In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Note: TensorFlow is not needed for PyNetwork to work. It's only used to load the dataset
import tensorflow as tf

import PyNetwork

import pyopencl as cl
import pyopencl.array as cl_array
from PyNetwork.gpu.GPUNN import GPUOPERATOR

2023-04-04 17:31:09.807257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

shape = (28, 28)
x_train = x_train.reshape(-1, *shape) / 255
x_test = x_test.reshape(-1, *shape) / 255

labels = np.eye(10)

y_train = labels[y_train.flatten()]
y_test = labels[y_test.flatten()]

# Large Learning Rate

In [25]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28)) )
model.add( PyNetwork.layers.Flatten() )
# model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0) )
# model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='relu', l2=0.0, l1=0.0) )

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0005)
model.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [26]:
%%time
model.train(x_train, y_train, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 1.4482 - accuracy: 0.7969
Training on 60000 samples
Epoch 2/10
cross_entropy: 1.3874 - accuracy: 0.8359
Training on 60000 samples
Epoch 3/10
cross_entropy: 1.2316 - accuracy: 0.8828
Training on 60000 samples
Epoch 4/10
cross_entropy: 1.1010 - accuracy: 0.8359
Training on 60000 samples
Epoch 5/10
cross_entropy: 0.5836 - accuracy: 0.8438
Training on 60000 samples
Epoch 6/10
cross_entropy: 1.0707 - accuracy: 0.8438
Training on 60000 samples
Epoch 7/10
cross_entropy: 0.7937 - accuracy: 0.8281
Training on 60000 samples
Epoch 8/10
cross_entropy: 0.7472 - accuracy: 0.8438
Training on 60000 samples
Epoch 9/10
cross_entropy: 1.4979 - accuracy: 0.8203
Training on 60000 samples
Epoch 10/10
cross_entropy: 1.0117 - accuracy: 0.8594
CPU times: user 17.8 s, sys: 910 ms, total: 18.8 s
Wall time: 3.46 s


In [24]:
model.evaluate(x_test, y_test)

'cross_entropy: 1.2138 - accuracy: 0.8370'

# Large Learning Rate with GPU

In [27]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)
gpuoperator = GPUOPERATOR(context=context, queue=queue)

model_gpu = PyNetwork.Sequential_GPU(context=context,queue=queue, gpuoperator=gpuoperator)

model_gpu.add( PyNetwork.layers.Input_GPU((28, 28)) )
model_gpu.add( PyNetwork.layers.Flatten_GPU(context=context, queue=queue, gpuoperator=gpuoperator) )
model_gpu.add( PyNetwork.layers.Dense_GPU(hidden_nodes=100, activation_function='relu', l2=0.01, l1=0.0,context=context, queue=queue, gpuoperator=gpuoperator) )
model_gpu.add( PyNetwork.layers.BatchNorm_GPU(context=context, queue=queue, gpuoperator=gpuoperator) )
model_gpu.add( PyNetwork.layers.Dense_GPU(hidden_nodes=10, activation_function='relu', l2=0.0, l1=0.0, context=context, queue=queue, gpuoperator=gpuoperator) )

/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/cache.py:114: UserWarning: could not obtain cache lock--delete '/Users/galen/Library/Caches/pyopencl/pyopencl-compiler-cache-v2-py3.10.9.final.0/lock' if necessary
  warn("could not obtain cache lock--delete '%s' if necessary"
/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/cache.py:516: UserWarning: PyOpenCL compiler caching failed with an exception:
[begin exception]
Traceback (most recent call last):
  File "/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/cache.py", line 493, in create_built_program_from_source_cached
    _create_built_program_from_source_cached(
  File "/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/cache.py", line 373, in _create_built_program_from_source_cached
    cache_result = retrieve_from_cache(cache_dir, cache_key)
  File "/Users/galen/opt/anaconda3/envs/HPC4M/lib/python3.10/site-packages/pyopencl/c

In [9]:
optimizer = PyNetwork.optimizers.RMSprop_GPU(context=context,queue=queue, gpuoperator=gpuoperator,learning_rate=0.0005)
model_gpu.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

AttributeError: 'tuple' object has no attribute 'get'

# Smaller Learning Rate

In [9]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28)) )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='relu', l2=0.0, l1=0.0) )

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0001)
model.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [10]:
%%time
model.train(x_train, y_train, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 1.6821 - accuracy: 0.5391
Training on 60000 samples
Epoch 2/10
cross_entropy: 1.3679 - accuracy: 0.6484
Training on 60000 samples
Epoch 3/10
cross_entropy: 1.1780 - accuracy: 0.7266
Training on 60000 samples
Epoch 4/10
cross_entropy: 0.9567 - accuracy: 0.7734
Training on 60000 samples
Epoch 5/10
cross_entropy: 0.8732 - accuracy: 0.7422
Training on 60000 samples
Epoch 6/10
cross_entropy: 0.9316 - accuracy: 0.7266
Training on 60000 samples
Epoch 7/10
cross_entropy: 0.7509 - accuracy: 0.7422
Training on 60000 samples
Epoch 8/10
cross_entropy: 0.7198 - accuracy: 0.7578
Training on 60000 samples
Epoch 9/10
cross_entropy: 0.5410 - accuracy: 0.8125
Training on 60000 samples
Epoch 10/10
cross_entropy: 0.5575 - accuracy: 0.8281
CPU times: user 2min 55s, sys: 9.12 s, total: 3min 5s
Wall time: 20.7 s


In [11]:
model.evaluate(x_test, y_test)

'cross_entropy: 0.6207 - accuracy: 0.8025'

# Test Part
